In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium 
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import json

In [ ]:
tiger_df = gpd.read_file('../data/tl_2021_47_tract.shp')
tiger_df = tiger_df.to_crs("EPSG:4326")

In [ ]:
type(tiger_df)

In [ ]:
davidson_loc = tiger_df.loc[(tiger_df.COUNTYFP == '037')]

poverty = pd.read_csv('../data/census_pov.csv')

In [ ]:
davidson_loc.head()

In [ ]:
poverty.head()

In [ ]:
poverty['GEO_ID'].str.split('S')

In [ ]:
poverty['GEO_ID'].str.split('S', expand = True)

In [ ]:
pov = poverty['GEO_ID'].str.split('S', expand = True)
poverty[['US_CODE', 'GEOID']] = pov

poverty.head()

In [ ]:
print('davidson loc shape')
print(davidson_loc.shape)
print('===================')
print('poverty shape')
print(poverty.shape)


In [ ]:
davidson_poverty = pd.merge(davidson_loc, poverty, on = "GEOID", how = "inner")

In [ ]:
davidson_poverty = davidson_poverty.rename(columns={"Total Population for whom poverty status os determined- Estimate":"poverty_est"})
davidson_poverty.dropna()

In [ ]:
davidson_poverty.plot()

In [ ]:
type(davidson_poverty)

In [ ]:
fig, ax = plt.subplots(figsize=(20,10))
davidson_poverty.plot(column = 'poverty_est', 
              cmap = 'GnBu', 
              edgecolor = 'black', 
              legend = True, 
              ax = ax)
ax.axis('off');

In [ ]:
davidson_poverty.plot(column = 'poverty_est', 
cmap = 'GnBu', 
edgecolor = 'black', 
legend = True)

In [ ]:
#Bring in crime data
crime = pd.read_csv('../data/police_incidents.csv')
crime.head()

In [ ]:
#metro_crime = gpd.read_file('..\data\metro_crime.geojson')
#metro_crime.head()

In [ ]:
davidson_loc.head()

In [ ]:
davidson_poverty['poverty_est'].max()

In [ ]:
m = folium.Map(location = [36.174465, -86.767960], zoom_start=10, control_scale=True)
m

In [ ]:
geoJson=davidson_loc.to_crs(epsg=4326).to_json()

folium.Choropleth(
    geo_data =geoJson,                  #json
    name ='choropleth',                  
    data = davidson_poverty,                     
    columns = ['TRACTCE', 'poverty_est'], #columns to work on
    key_on ='feature.properties.TRACTCE',
    fill_color ='YlGnBu',     #I passed colors Yellow,Green,Blue
    fill_opacity = 0.7,
    line_opacity = 0.2,
    legend_name = "Total Population for whom poverty status is determined- Estimate"
).add_to(m)

m

In [ ]:
for _, r in davidson_poverty.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    dav_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = dav_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    geo_j.add_to(m)
    
m